# 03. Data Preprocessing


## 1. Objective
- Làm sạch dữ liệu
- Chuẩn bị cho bước kỹ thuật tính năng

## 2. Missing Value Handling
- Phân tích nguyên nhân
- Forward fill / interpolation

## 3. Outlier Analysis
- Phát hiện ngoại lệ
- Nhận xét: ngoại lệ là sự kiện thật hay nhiễu

## 4. Time Index Processing
- Chuẩn hóa datetime
- Sắp xếp theo thời gian

## 5. Scaling (If Needed)
- Standardization / Normalization
- Lý do lựa chọn

## 6. Train-Test Split Strategy
- Time-based split
- Tránh data leakage


## 7. Conclusion
- Dữ liệu sẵn sàng cho feature engineering